In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('vaccine_preprocessed.csv',sep='|')

In [3]:
df.head(1)

,categories,titles,contents,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,urls
0,AZ,澎湖確診數升 兒童疫苗接種率已逾7成,澎湖COVID-19（2019冠狀病毒疾病）確診數攀升，澎湖縣政府今天公布新增182例，還有...,0.0,"['澎湖5至11歲兒童疫苗接種種率已逾7成', '澎湖縣政府今天公布新增182例確診案例',...","[('澎湖', 5), ('疫苗', 5), ('接種', 4), ('確診', 3), (...","['澎湖', 'COVID-19', '（', '2019', '冠狀', '病毒', '疾...","['澎湖', '冠狀', '病毒', '疾病', '確診數', '澎湖縣', '政府', '...","[NerToken(word='澎湖', ner='GPE', idx=(0, 2)), N...","[('澎湖', 'Nc'), ('COVID-19', 'FW'), ('（', 'PARE...",https://news.google.com/articles/CBMiMmh0dHBzO...


In [40]:
# Searching keywords from "token_v2" column
def filter_dataFrame(user_keywords, cond, cate):

    # proceed filtering
    if (cate == "全部") & (cond == 'and'):
        query_df = df.tokens_v2.apply(lambda text: all((qk in text) for qk in user_keywords))
    elif (cate == "全部") & (cond == 'or'):
        query_df = df.tokens_v2.apply(lambda text: any((qk in text) for qk in user_keywords))
    elif (cond == 'and'):
        query_df = df[(df.categories == cate)  
            & df.tokens_v2.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cond == 'or'):
        query_df = df[(df.ccategories == cate) 
            & df.tokens_v2.apply(lambda text: any((qk in text) for qk in user_keywords))]
    
    return query_df

In [41]:
user_keywords=['烏克蘭','台灣']
cond='and'
cate='AZ'
df_query = filter_dataFrame(user_keywords, cond, cate)

In [42]:
user_keywords=['烏克蘭','台灣']
cond='and'
cate='AZ'


In [43]:
query_df = df[(
    df.categories == cate)   # category
    & df.tokens_v2.apply(lambda row: all((qk in row) for qk in user_keywords)) # user keywords
    ]
len(query_df)

1

In [44]:
user_keywords=['烏克蘭','台灣']
cond='and'
cate='全部'

In [45]:
if (cate == "全部") & (cond == 'and'):
    query_df = df[
        df.tokens_v2.apply(lambda row: all((qk in row) for qk in user_keywords))
        ]
elif (cate == "全部") & (cond == 'or'):
    query_df = df[
        df.tokens_v2.apply(lambda row: any((qk in row) for qk in user_keywords))]

In [46]:
len(query_df)

3

In [10]:
# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_dataFrame_fullText(user_keywords, cond, cate):

    # proceed filtering
    if (cate == "全部") & (cond == 'and'):
        df_query = df.content.apply(lambda text: all((qk in text) for qk in user_keywords))
    elif (cate == "全部") & (cond == 'or'):
        df_query = df.content.apply(lambda text: any((qk in text) for qk in user_keywords))
    elif (cond == 'and'):
        df_query = df[(df.categories == cate)  
            & df.content.apply(lambda text: all((qk in text) for qk in user_keywords))]
    elif (cond == 'or'):
        df_query = df[(df.categories == cate) 
            & df.content.apply(lambda text: any((qk in text) for qk in user_keywords))]

    return df_query

In [12]:
user_keywords=['陳時中','疫苗']
cond='and'
cate='全部'
df_query = filter_dataFrame(user_keywords, cond, cate)
len(df_query)

796

In [31]:
user_keywords=['陳時中','疫苗']
cond='and'
cate='AZ'
df_query = filter_dataFrame(user_keywords, cond, cate)
len(df_query)

57

In [37]:
df_query.head(1)

,categories,titles,contents,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,urls
4,AZ,AZ疫苗第2劑本週開放50至64歲預約 可望雙十連假前開打,指揮中心今天公布，AZ疫苗第2劑改由線上預約，即起開放9月30日前間隔滿10週以上50歲至6...,0.0,['中央流行疫情指揮中心原規劃COVID-19（2019冠狀病毒疾病）疫苗第10輪10月3日...,"[('疫苗', 12), ('預約', 8), ('接種', 7), ('意願', 6), ...","['指揮', '中心', '今天', '公布', '，', 'AZ', '疫苗', '第2'...","['指揮', '中心', '疫苗', '線上', '預約', '開放', '間隔', '民眾...","[NerToken(word='今天', ner='DATE', idx=(4, 6)), ...","[('指揮', 'Na'), ('中心', 'Nc'), ('今天', 'Nd'), ('公...",https://news.google.com/articles/CBMiN2h0dHBzO...


In [38]:
# For the query_df, count the occurence and frequency for each category.

# (1) cate_occurence={}  被多少篇新聞報導 How many pieces of news contain the keywords.
# (2) cate_freq={}       被提到多少次? How many times are the keywords mentioned


vaccine_categories = ['AZ', 'BNT', '莫德納', '高端', '全部']

def count_keyword(query_df, user_keywords):
    cate_occurence={}
    cate_freq={}

    for cate in vaccine_categories:
        cate_occurence[cate]=0
        cate_freq[cate]=0

    for idx, row in query_df.iterrows():
        # count number of news
        cate_occurence[row.categories] += 1
        cate_occurence['全部'] += 1
        
        # count user keyword frequency by checking every word in tokens_v2
        tokens = eval(row.tokens_v2)
        freq =  len([word for word in tokens if (word in user_keywords)])
        cate_freq[row.categories] += freq
        cate_freq['全部'] += freq
        
    return cate_freq, cate_occurence

In [48]:
user_keywords=['陳時中','疫苗']
count_keyword(df_query, user_keywords)

({'AZ': 2, 'BNT': 0, '莫德納': 0, '高端': 0, '全部': 2},
 {'AZ': 1, 'BNT': 0, '莫德納': 0, '高端': 0, '全部': 1})